# Neural Affect Style Transfer

In [1]:
from numpy import zeros, concatenate, asarray, ones, amax, argmax
from IPython.display import display, HTML

In [2]:
def browser_alert(message):
    display(HTML('<script type="text/javascript">alert("' + message + '");</script>'))
    
def browser_notify(message):
    display(HTML('<script type="text/javascript">var notification=new Notification("' + \
                 'Jupyter Notification",{icon:"http://blog.jupyter.org/content/' + \
                 'images/2015/02/jupyter-sq-text.png",body:"' + message + \
                 '"});</script>'))

In [3]:
browser_notify("test")

## Read Data

In [4]:
dataset_path = "/home/v2john/attr-reviews-dataset/dev.txt"

In [5]:
all_texts = list()
with open(dataset_path) as dataset_file:
    for line in dataset_file:
        text = line.split('\t')[3]
        all_texts.append(text)

In [6]:
len(all_texts)

93703

### Tokenize and build embeddings

In [7]:
from keras.preprocessing.text import Tokenizer

Using TensorFlow backend.


In [8]:
keras_tokenizer = Tokenizer(char_level=True)

In [9]:
keras_tokenizer.fit_on_texts(all_texts)

In [10]:
len(keras_tokenizer.word_index)

73

In [11]:
text_sequences = keras_tokenizer.texts_to_sequences(all_texts)

In [12]:
text_sequences[1]

[5,
 14,
 5,
 4,
 17,
 2,
 1,
 8,
 3,
 2,
 5,
 17,
 15,
 1,
 10,
 4,
 3,
 1,
 9,
 2,
 5,
 12,
 1,
 29,
 1,
 11,
 4,
 24,
 2,
 12,
 1,
 6,
 3,
 1,
 29,
 25]

In [13]:
MAX_SEQUENCE_LENGTH = max(len(sequence) for sequence in text_sequences)

In [14]:
MAX_SEQUENCE_LENGTH

367

## Keras Model

In [15]:
from keras.layers import Input, Dense, RepeatVector, LSTM, Conv1D, Masking, Embedding
from keras.layers.wrappers import TimeDistributed, Bidirectional
from keras.models import Model
from keras.preprocessing.sequence import pad_sequences

In [16]:
VOCAB_SIZE = len(keras_tokenizer.word_index) + 1

In [17]:
x_train = pad_sequences(text_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post', 
                        truncating='post', value=0)

In [18]:
x_train = x_train[:5000]

In [19]:
x_train.shape

(5000, 367)

In [20]:
x_train_rev = list()
for x_vector in x_train:
    x_rev_vector = list()
    for index in x_vector:
        char_vector = zeros(VOCAB_SIZE)
        char_vector[index] = 1
        x_rev_vector.append(char_vector)
    x_train_rev.append(asarray(x_rev_vector))
x_train_rev = asarray(x_train_rev)

In [33]:
x_train_rev.shape

(5000, 367, 74)

In [34]:
main_input = Input(shape=x_train_rev.shape[1:], dtype='float32', name='main_input')
print(main_input)

# embed_1 = Embedding(input_dim=VOCAB_SIZE, output_dim=EMBEDDING_DIM, 
#                     mask_zero=True, input_length=MAX_SEQUENCE_LENGTH) (main_input)
# print(embed_1)

# lstm_1 = Bidirectional(LSTM(EMBEDDING_DIM, return_sequences=True, name='lstm_1'))(embed_1)
# print(lstm_1)

lstm_2 = Bidirectional(LSTM(100, name='lstm_2'))(main_input)
print(lstm_2)

repeat_1 = RepeatVector(MAX_SEQUENCE_LENGTH, name='repeat_1')(lstm_2)
print(repeat_1)

lstm_3 = Bidirectional(LSTM(100, return_sequences=True, name='lstm_3'))(repeat_1)
print(lstm_3)

# lstm_4 = LSTM(EMBEDDING_DIM, return_sequences=True, name='lstm_4')(lstm_3)
# print(lstm_4)

softmax_1 = TimeDistributed(Dense(VOCAB_SIZE, activation='softmax'))(lstm_3)
print(softmax_1)

Tensor("main_input_1:0", shape=(?, 367, 74), dtype=float32)
Tensor("bidirectional_3/concat_2:0", shape=(?, 200), dtype=float32)
Tensor("repeat_1_1/Tile:0", shape=(?, 367, 200), dtype=float32)
Tensor("bidirectional_4/concat_2:0", shape=(?, ?, 200), dtype=float32)
Tensor("time_distributed_2/Reshape_1:0", shape=(?, 367, 74), dtype=float32)


In [43]:
model = Model(main_input, softmax_1)
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [44]:
model.fit(x_train_rev, x_train_rev, batch_size=32, epochs=50, verbose=1)

Epoch 1/50
5000/5000 [==============================] - 267s - loss: 0.8097 - acc: 0.7732   
Epoch 2/50
5000/5000 [==============================] - 266s - loss: 0.7932 - acc: 0.7742   
Epoch 3/50
5000/5000 [==============================] - 266s - loss: 0.7923 - acc: 0.7740   
Epoch 4/50
5000/5000 [==============================] - 266s - loss: 0.7873 - acc: 0.7753   
Epoch 5/50
5000/5000 [==============================] - 266s - loss: 0.7878 - acc: 0.7752   
Epoch 6/50
5000/5000 [==============================] - 266s - loss: 0.7858 - acc: 0.7754   
Epoch 7/50
5000/5000 [==============================] - 266s - loss: 0.7836 - acc: 0.7759   
Epoch 8/50
5000/5000 [==============================] - 266s - loss: 0.7859 - acc: 0.7753   
Epoch 9/50
5000/5000 [==============================] - 266s - loss: 0.7816 - acc: 0.7764   
Epoch 10/50
5000/5000 [==============================] - 266s - loss: 0.7803 - acc: 0.7767   
Epoch 11/50
5000/5000 [==============================] - 266s - loss:

In [45]:
predictions = model.predict(x_train_rev)

In [46]:
char_dict = dict()
for char in keras_tokenizer.word_index:
    rank = keras_tokenizer.word_index[char]
    if rank <= VOCAB_SIZE:
        char_dict[rank] = char

In [47]:
max_prob = 0.3
def sequence_to_str(sequence):
    char_list = list()
    for element in sequence:
#         if amax(element) < max_prob:
#             continue        
        index = argmax(element) + 1
        char = char_dict[index]
        char_list.append(char)
        
    return char_list

In [48]:
for i in range(len(predictions)):
    predicted_char_list = sequence_to_str(predictions[i])
    actual_len = len(all_texts[i])
    print("Actual: " + all_texts[i])
    print("Generated: " + "".join(predicted_char_list[:actual_len]) + " - " + "\n")

Actual: i loved this book , it was touching

Generated: neda
tueolnrefaapeeeeeeeeeeeeeeeeeee - 

Actual: awaome steamy hot read ! loved it !

Generated: iei

teeeeetteeeeeeeeeeeeeeeeeettexx - 

Actual: loved this book.lisa gardner never disappoints me !

Generated: da
tueolnrefaapeeeeeeeeetttttiteeeeeeeeeeeeeeeeeee,  - 

Actual: i liked it but want to know more

Generated: nednptueooeeeaeeeeeeeeeeeeeeeaeee - 

Actual: campaigning , lying , cold blooded garbage .

Generated: ii..ssssmmmmseeeeeeeeeeeeeeeeeeeeeeeeeeeeeeb, - 

Actual: ca n't wait for the next one .

Generated: i..oooooeeeoooeeeeeetteeeeeeee, - 

Actual: enjoyed this book . i would recommend .

Generated: ts!agtueolnrefaapebeeeeeeeeeeeeeeeeeeeb, - 

Actual: excellent .contniue the story line of tucker series

Generated: t...ddtsooeateeeooooooeeeeeeeeeeeeeeeeeeeeeeeeetteeb - 

Actual: great book . read everyone of her books

Generated: mhtioefaapeittttttttttttttaaaaaaaaaaaaeb - 

Actual: the writing was disjointed hard to ge


Generated: da
tueolteeoeeeeeeeeeddddddddddddddeeeeeeeeeeeeeeeeeeeeeeeeeeeettttte,  - 

Actual: this was probably the best book in the series . # teamkeys

Generated: olnreyreeeaaaeeaaeeeeooooeffaaaeeeeeeeeeeeeeeeeeeeeeetttee, - 

Actual: very enjoyable . looking forward to the next in the series .

Generated: 
ttgestaageeaaeeeaeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeetttte, - 

Actual: it was a cute book and for free it was not bad

Generated: noeyireiesoteefaapeeeeeeeetttteeeeeeeeeeeeeee,, - 

Actual: i love all leighann dobbs books with lexy and her friends !

Generated: neda
tedddeeeiiitsseeeeeeefaaeeeeeeeeeeeeeeeeeeeeeeeetttte,, - 

Actual: a great read and a very poignant story . loved it !

Generated: iemhhgeeiiiueiiiueeuuueeeeeeeeeeeeeeeeeeeeeeeeeeeeb, - 

Actual: loved those kids and their bravery in the life they had .

Generated: da
tueonreeaaeeeeeeeeettttttttteeeeeeeeeeeeeeeeeeettttte,, - 

Actual: it 's been too long . i ca n't leave a review

Generated: nnerretttteeeaaaeeeeee


Actual: very good book - i read all the dresden files with bated breath.linda sheean

Generated: 
thgemaauefaapeeeeeeeeeeettttttttteeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeettee, - 

Actual: again for my wife and she loves the irish and scottish type books .

Generated: iiiieesaaeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeetttteeeeeeeeeeeeeeeettteeb, - 

Actual: beck writes well , and his stories are credible as are his characters .

Generated: naateetleeeeeddddddeeeellleeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeetttee,, - 

Actual: the construct of the story is quite elaborate , but a fun read .

Generated: olte.aateeeeeeeoooooeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeettteb, - 

Actual: excellent book held my attention . i will read more by this author .

Generated: ttttddtsoefaapeeeeeeeeeettttteeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeb, - 

Actual: light , easy , fun reading . i enjoyed this series a lot .

Generated: noolrrrrreeeeeeeeeeeeeeetttttttteeeeeeeeeeeeeeeeeeeeeteeeb, - 

Actual: great book - wonderfu


Generated: iaheeooeooooeeaaeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeteee,  - 

Actual: remained true to characters and kept my interest.enjoyed the story and would recommend this book to mymom.enjoy

Generated: ititssssseeeeeeeeeeeotttttteeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeaaee,, - 

Actual: examine your faith up close and personal with this in-depth study of the evolution of religion .

Generated: i...sssssaaeeeeoeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeetteeb, - 

Actual: would recommend ! i really like this story ! cannot wait to read the next book !

Generated: yaauueteesttteeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeexx  - 

Actual: what can i say but i loved it and wish it did n't have to end !

Generated: ylioeennnneeeeeaeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeetteex, - 

Actual: it 's one of the only books in my life i literally could not get interested in .

G


Generated: neda
teolnrefaapeneeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee,  - 

Actual: very petty . could n't get into it . did n't find it the least bit funny .

Generated: tttgeooooeeeaaeeeeeeeeooooooooeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeb, - 

Actual: great writersteamy storiesthe entire rescue me series is a must readplease keep em commingmore more morei am hooked

Generated: mhtiooohttorrrttteeeooooooooootttttttttttttttteeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeetatee,  - 

Actual: like the way it was written and the authorstory line was good to the periodwould recommend to friends

Generated: ndpteoleeeyyeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeaaee,, - 

Actual: i read it too long ago to remember the plot , but i always like her books .

Generated: nehtiueooeeaaaaeeeeoeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeetttee,  - 

Actual: i enjoyed reading this book kept me in suspenset

...

Generated: tthddgedtatueihhiiiiiihhhheeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee,  - 

Actual: good reading from the start and gets better as you go . how often is it someone who it should n't be

Generated: maauehtueesseeeeeeooooeeeeeeeetteeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeatteb, - 

Actual: found the book not for me . too much like other books of its elk . perhaps a younger age group .

Generated: aawuueoltefaapeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeetee,,  - 

Actual: daily scripture reading and meditation : <num> healing bible verses - .really am enjoying this wonderful book . thank you from jan

Generated: iiioeennetttttttttttuuutttttteeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee,  - 

Actual: i ca n't wait for the movie to come out just to see how much of the book they put in it

Generated: ne.issoooooooooo

Generated: ireiiideeeeeeeeeeeeeeeeeeeeeeeettttteeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeettteb,  - 

Actual: i did not like this book and the way it was written . i started it and then gave it up .

Generated: nennueeneeepuueoolneffapeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee, - 

Actual: this adds a little dimension to the outlander series . it was a fun trip into the life of lord john .

Generated: olneeitueieeeeeeeeeeeeeeeoooeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeetteeb,  - 

Actual: this book is truly a great novel as are all books written by beth wiseman . i enjoy anthing she writes .

Generated: olnrefaapeyrreeaaeeeeeeettttteeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeettee,,  - 

Actual: a very good book kept your interest through out the whole thing was a very good short reading book kept your interest

Generated: ietthgemaauefaapeaaeeeeeeeeetttooooooooooooeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee

Generated: olnrefaapeyieeeeeeaaaaaeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee,  - 

Actual: wow cassia out did herself . the suspense was amazing i couldnt put the book down . cannot wait for volumn <num> .

Generated: yydeeereeeeateeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeaaee,   - 

Actual: really enjoyed the entire storyline , not really being able to know what 's going to happen next . very good reading .

Generated: ttiddgets!aggueoltetotttrreeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeaaee,   - 

Actual: not my thing ? but you never know you might like it . i am sad to say that i did not .

Generated: ssseweooooeeoooaaaeeeeeeeeeeeeeeaaaaaaeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeb,  - 

Actual: her books are fast paced , fun and exiting , you will fall in love with rachel one of the main caracters .

Generated: ottefaaaeyiieeeeeeeeeeeeeeeeeeeeeeeeeeeeee

Actual: lesley pearce never disappoints , love all her books.really enjoyed this story and read it until late at night to finish it .

Generated: dtddddtttttttttttttttteerrrreeeeeedeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeettte,,   - 

Actual: these were great stories as that had a story as well as love , sex and a bit of happy ever after .

Generated: oltehttttttttttoooeeeeeeeeeeeeeetttteeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeetteeb, - 

Actual: predictable as all romance stories are . needed an inexpensive book to read for a slow day and this fit the bill .

Generated: httttetiiiiedddddddeeeeeeeeeeeeeeeeeeetttttttteeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeettee,,  - 

Actual: the book was the terrific ending to a sensational love story . i smiled and sighed and loved every minute if it .

Generated: oltefaapeyyeeelttttttttteeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeetteee


Generated: ttiddgemaaueeeaaeeeeeeeeeeeeeetttttteeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeetttte,   - 

Actual: a quirky read . a new and definitely delightful heroine . a light and fanciful story with an unusual cast of characters .

Generated: ieewwtuetueeeeeeeeeeeeeeeeeeedddddeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeettttee,   - 

Actual: great series loved them all . very good story kept me interested and wanting to read more . she is a good writer

Generated: mhtiootttttttttttttttttttteeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeettee,, - 

Actual: this is an enjoyable novel but became redundant over time and chapters . some of the characters were very endearing . enjoy !

Generated: olnrenreeessttgaaaaeaaaaaaaeeeeeeettttttttttttttttttttttteeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeetttttee,,    - 

Actual: this book was a fun , easy read and definitely like the way this a

In [49]:
browser_notify("Sentences generated")